# Inversion of a linear regressor

In [1]:
import numpy as np
from regressor import LinearRegressor

import inversion_ideas as ii

## Synthetic data

Build synthetic data for a linear regressor such as:

$$ \mathbf{d}_\text{obs} = \mathbf{X} \mathbf{m}_\text{true} + \epsilon $$

where 
$\mathbf{d}_\text{obs}$ is the observed data (synthetic),
$\mathbf{X}$ is a matrix we randomly generate,
and $\mathbf{m}_\text{true}$ is the true model we'll try to invert for,
and $\epsilon$ is some random noise generated out of a normal distribution.

In [2]:
n_params = 10
rng = np.random.default_rng(seed=4242)
true_model = rng.uniform(size=10)
true_model

array([0.78225148, 0.67148671, 0.2373809 , 0.17946133, 0.34662367,
       0.15210999, 0.31142952, 0.23900652, 0.54355731, 0.91770851])

In [3]:
# Build the X matrix
n_data = 25
shape = (n_data, n_params)
X = rng.uniform(size=n_data * n_params).reshape(shape)

In [4]:
# Generate synthetic data with noise
synthetic_data = X @ true_model
maxabs = np.max(np.abs(synthetic_data))
std_err =  1e-2 * maxabs
noise = rng.normal(scale=std_err, size=synthetic_data.size)
synthetic_data += noise
synthetic_data

array([2.83840696, 2.18091081, 2.00623242, 2.08333039, 2.01694883,
       2.7826232 , 2.10564027, 1.27333506, 2.08859855, 1.94177648,
       1.88492037, 2.92394733, 2.17231952, 3.08009275, 1.61670886,
       1.77403753, 2.67305005, 1.91413882, 2.42117827, 2.13991628,
       2.0153805 , 2.71388471, 2.65944255, 2.44416121, 3.14217523])

## Inversion

In [5]:
simulation = LinearRegressor(X)

In [6]:
uncertainty = std_err * np.ones_like(synthetic_data)
data_misfit = ii.DataMisfit(synthetic_data, uncertainty, simulation)
data_misfit

φd(m)

In [7]:
smallness = ii.TikhonovZero(n_params)
smallness

φs(m)

In [8]:
phi = data_misfit + 1e-3 * smallness
phi

φd(m) + 0.00 φs(m)

In [9]:
initial_model = np.zeros(n_params)
initial_model

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [10]:
from scipy.optimize import minimize

result = minimize(phi, initial_model)
result

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 11.719746150008552
        x: [ 8.133e-01  6.593e-01  2.473e-01  1.963e-01  3.237e-01
             1.472e-01  3.195e-01  2.524e-01  5.222e-01  9.218e-01]
      nit: 15
      jac: [-5.841e-06 -2.384e-07 -3.576e-06 -2.384e-07 -7.987e-06
            -1.311e-06  0.000e+00  1.907e-06 -2.503e-06 -5.603e-06]
 hess_inv: [[ 3.378e-04 -1.043e-04 ...  1.536e-05 -9.322e-05]
            [-1.043e-04  4.045e-04 ... -1.009e-04 -1.335e-05]
            ...
            [ 1.536e-05 -1.009e-04 ...  2.159e-04 -5.467e-05]
            [-9.322e-05 -1.335e-05 ... -5.467e-05  2.625e-04]]
     nfev: 253
     njev: 23

In [11]:
inverted_model = result.x
inverted_model

array([0.8132614 , 0.65925412, 0.24726513, 0.19626776, 0.32373026,
       0.14719295, 0.3194738 , 0.25235066, 0.52216838, 0.92184153])

In [12]:
print("Result:")
print(inverted_model)
print()
print("True model:")
print(true_model)

Result:
[0.8132614  0.65925412 0.24726513 0.19626776 0.32373026 0.14719295
 0.3194738  0.25235066 0.52216838 0.92184153]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]


### Use the `Minimizer` class

In [13]:
minimizer = ii.ConjugateGradient()
minimizer

In [14]:
inverted_model = minimizer(phi, initial_model)
inverted_model

array([0.81328886, 0.65927514, 0.2472937 , 0.19624752, 0.3237346 ,
       0.14720342, 0.3194468 , 0.25235983, 0.52215485, 0.92181018])

In [15]:
print("Result:")
print(inverted_model)
print()
print("True model:")
print(true_model)

Result:
[0.81328886 0.65927514 0.2472937  0.19624752 0.3237346  0.14720342
 0.3194468  0.25235983 0.52215485 0.92181018]

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]
